In [ ]:
#pip install minio

In [1]:
import pandas as pd
import pyspark as ps
import glob
import os
import io
import pathlib
from minio import Minio

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark import SparkContext, SparkConf

In [ ]:
spark = SparkSession.builder \
  .master("local") \
  .appName("parquet_example") \
  .getOrCreate()

In [2]:
ACCESS_KEY='tensorflow'
SECRET_KEY='tensorflow'

In [3]:
client = Minio(
       '172.25.0.4:9000',
        access_key=ACCESS_KEY,
        secret_key=SECRET_KEY,
        secure=False
    )

In [4]:
bucket_name = 'landing'
bucket_destination = 'temp'
parque_folder_path = 'parquet_saved'

In [5]:
def extract():  
    #busca a lista de objetos no data lake.
    objects = client.list_objects('landing', recursive=True)
    
    #faz o download de cada arquivo e concatena com o dataframe vazio.
    for obj in objects:
        print("Downloading file...")
        print(obj.bucket_name, obj.object_name)

        client.fget_object(
                    obj.bucket_name,
                    obj.object_name,
                    'temp/'+obj.object_name,
        )

In [6]:
#executa a funcao para fazer o download dos arquivos
extract()

landing olist_customers_dataset.csv
landing olist_geolocation_dataset.csv
landing olist_order_items_dataset.csv
landing olist_order_payments_dataset.csv
landing olist_order_reviews_dataset.csv
landing olist_orders_dataset.csv
landing olist_products_dataset.csv
landing olist_sellers_dataset.csv
landing product_category_name_translation.csv


In [7]:
def load():
    #carrega os nomes de arquivos a partir de uma pasta.
    filename = os.listdir('temp')
    
    
    for obj in filename:
        #gera os datasets
        df = pd.read_csv("temp/"+obj, sep=',', header=0)
        #tira a extensao do nome do arquivo
        filenamemod = pathlib.Path(obj).stem
        #converte o dataset em parquet
        df.to_parquet("parquet_saved/{}.parquet".format(filenamemod),index=False)

        #carrega os dados para o Data Lake.
        client.fput_object("processing",filenamemod+".parquet","parquet_saved/{}.parquet".format(filenamemod))

In [8]:
load()